## Neural Networks

- 신경망은 torch.nn 패키지를 사용하여 생성할 수 있다. 
- nn은 모델을 정의하고 미분하는데 autograd를 사용한다.
- nn.Module 은 계층과 output을 반환하는 forward(input) 메서드를 포함한다. 

신경망의 일반적인 학습 과정은 다음과 같다. 
1. 학습 가능한 매개변수(가중치 weight)를 갖는 신경망을 정의한다.
2. 데이터셋 입력을 반복한다.
3. 입력을 신경망에서 전파한다.
4. loss을 계산한다.
5. gradient를 신병망의 매개변수들에 역으로 전파한다.
6. 신경망의 가중치를 갱신한다. 
7. 가중치(wiehgt) = 가중치(weight) - 학습율(learning rate) * 변화도(gradient)

## 신경망 정의하기


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        # kernel
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        # an affine operation
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)
        
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x= self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *=s
        return num_features


In [2]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


- forward 함수만 정의하고 나면, backward 함수는 autograd를 사용하여 자동으로 정의된더.
- 모델의 학습 가능한 매개변수들은 net.parameters()에 의해 반환된다

In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's.weight

10
torch.Size([6, 1, 3, 3])


- 임의의 32x32 입력값을 넣는다. 
- 이 신경망(LeNet)의 예상되는 입력 크기는 32x32이다.
- 이 신경망에 MNIST 데이터셋을 사용하기 위해선, 데이터셋의 이미지 크기를 32x32로 변경해야한다. 


In [4]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0640, -0.0884,  0.0309,  0.0578, -0.0418,  0.0080, -0.0367, -0.0406,
          0.0125,  0.1298]], grad_fn=<AddmmBackward>)


모든 매개변수의 변화도 버퍼를 0으로 설정하고, 무작위 값으로 역전파를 한다.

In [5]:
net.zero_grad()
out.backward(torch.randn(1,10))

- torch.nn은 미니 배치(mini-batch)만 지원한다. torch.nn 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니-배치만을 입력으로 받는다. 
- torch.Tensor - backward() 같은 autograd 연산을 지원하는 다차원 배열.
- nn.Module - 신경망 모듈. 매개변수를 캡슐화(encapsulation)하는 간편한 방법 으로, GPU로 이동, 내보내기(exporting), 불러오기(loading) 등의 작업을 위한 헬퍼(helper)를 제공한다.
- nn.Parameter - Tensor의 한 종류로, Module에 속성으로 할당될 때 자동으로 매개변수로 등록된다. 
- autograd.Function - autograd 연산의 전방향과 역방향 정의를 구현한다. 
- 모든 Tensor 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Tensor를 생성하고 history를 부호화하는 함수들과 연결한다.


## Loss Function


- 손실 함수는 (output, target)을 한 쌍(pair)의 입력으로 받아, 출력(output)이 정답(target)으로부터 얼마나 멀리 떨어져있는지 추정하는 값 계산.
- nn 패키지에는 여러 손실 함수들이 존재
- 간단한 손실 함수로는 출력과 대상간 평균 제곱 오차(mean-squared error)를 계산하는 nn.MSEloss가 있다.

In [6]:
output = net(input)
target = torch.randn(10) # a dummy target
target = target.view(1,-1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5806, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # Relu

## 역전파(BACKPROP)


- 오차를 역전파하기 위해서는 loss.backward()만 해주면 된다. 
- 기존 변화도를 없애는 작업이 필요한데, 그렇지 않으면 변화도가 기존의 것에 누적이 되기 때문이다. 


loss.backward() 를 호출하여 역전파 전과 후에 conv1과 bias gradient를 비교

In [8]:
net.zero_grad() # zeros the gradient bufferrs of all parameterss

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0028,  0.0004, -0.0030, -0.0048,  0.0021, -0.0174])


## 가중치 갱신

실제로 많이 사용되는 가장 단순한 갱신 규칙은 확률적 경사 하강법(SGD)이다. 

- 가중치(wiehgt) = 가중치(weight) - 학습율(learning rate) * 변화도(gradient)

In [9]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)


- torch.optim 라는 작은 패키지에 이러한 방법들을 모두 구현해되어있다.

In [10]:
import torch.optim as optim

# Optimizerr 생성
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# 학습 과정
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Does the update